# Geospatial Analysis of Agricultural Parcels with Fertilization Treatments

*Author: Alessandro Jopshua Pierro*

*Affiliation: Agrscope*

*Date: December 2023*

## Overview
This Jupyter notebook is a comprehensive tool for visualizing and analyzing agricultural parcels. It focuses on displaying the variations in fertilization treatments across parcels using advanced geospatial data processing techniques. The notebook generates interactive maps that are not only insightful but also user-friendly, with outputs available in HTML and PNG formats.
## Objective
The primary goal is to create a visual representation of different fertilization regimes applied to agricultural parcels. This visualization aids in understanding how various treatments are distributed across the landscape and facilitates the evaluation of their impact on agricultural productivity.
## Methodology
The notebook leverages the create_map function to process geographical data from shapefiles. It centers maps on the parcels and styles them based on their fertilization treatments. The function generates maps in both HTML for interactive use and PNG for static use.
## Key Functions
create_map: Reads geographical data from shapefiles, creates Folium maps centered on the parcels, and applies styling based on fertilization treatments.

## Required Libraries and Dependencies

Below are the necessary libraries and modules required for the geospatial analysis and visualization. Ensure these are installed in your Python environment before running this notebook.

In [6]:
import time
import warnings
import os
import folium
import io
import geopandas as gpd
from pathlib import Path
from PIL import Image
from selenium import webdriver

## Map Creation Function

The `create_map` function below is responsible for generating maps based on shapefiles found in the specified directory. It reads the geographical data, creates a Folium map centered on the parcels, and styles each parcel based on its fertilization treatment.

In [30]:
def create_map(folder_name):
    """
    Generates HTML and PNG maps for the given parcel directory. Maps are centered on the parcels and display fertilization treatments.
    
    Parameters:
    folder_name (str): The name of the folder containing the parcel shapefiles.
    """
    # Ignore warnings
    warnings.filterwarnings('ignore')
    
    # Define base directory
    base_directory = Path(os.path.dirname(os.path.realpath("__file__")))

    # Construct the shapefile path
    shapefile_name = f'{folder_name}.shp'
    shapefile_path = base_directory.joinpath('field_information/Shapefiles', folder_name, shapefile_name)

    # Read parcel and calculate its centroid for setting map center properly
    parcel = gpd.read_file(shapefile_path)
    parcel_wgs84 = parcel.to_crs('EPSG:4326')  # Convert to geographic coordinates (WGS84, EPSG:4326)
    parcel_centroid = parcel_wgs84.geometry.centroid

    # Create map centered at the mean centroid with an adjusted zoom level
    map_center = [parcel_centroid.y.mean(), parcel_centroid.x.mean()]
    # Decrease the zoom_start value if the map is too zoomed in
    map = folium.Map(location=map_center, zoom_start=19, tiles="Stamen Terrain")  # Try different zoom levels


    # Adjust title and subtitle styles for better visibility
    title_html = '''
        <h3 align="center" style="font-size:20px; margin-bottom:0; padding: 10px; background-color: white; opacity: 0.8;"><b>Parcel of Interest and their Fertilization</b></h3>
    '''
    subtitle_html = '''
        <p align="center" style="font-size:16px; margin-top:0; padding: 5px; background-color: white; opacity: 0.8;">{}</p>
    '''.format(folder_name)
    map.get_root().html.add_child(folium.Element(title_html))
    map.get_root().html.add_child(folium.Element(subtitle_html))
    
    # Define color mapping for 'Traitement' and add more if necessary
    color_mapping = {
        '0N': '#66c2a5',
        'Nmin': '#fc8d62',
        'Standard': '#8da0cb',
        '0N_2': '#66c2a5',
        'Nmin_2': '#fc8d62',
        'Standard_2': '#8da0cb'
    }

    # Function to set style based on 'Traitement' column
    def set_style(feature):
        traitement = feature['properties']['Traitement']
        color = color_mapping.get(traitement, '#cccccc')  # Default to gray if not in mapping
        return {'fillColor': color, 'color': color, 'weight': 2, 'fillOpacity': 0.7}

    # Function to display 'Traitement' information in popup
    def set_popup(feature):
        traitement = feature['properties'].get('Traitement', 'N/A')
        return f'Traitement: {traitement}'

    # Add GeoJson layer with customized style and popup
    folium.GeoJson(
        data=parcel,
        style_function=set_style,
        tooltip=folium.GeoJsonTooltip(fields=['Traitement'], aliases=['Traitement'], labels=True)
    ).add_to(map)

    # Add labels above the parcels
    for idx, centroid, traitement in zip(parcel.index, parcel_centroid, parcel['Traitement']):
        # Determine label size dynamically based on text length
        label_size = str(len(traitement) * 8)  # This is a simplistic approach; adjust as needed
        label_width = len(traitement) * 8  # Estimate width based on character count
        label_height = 20  # An estimated height for the label
        folium.map.Marker(
            [centroid.y, centroid.x],
            icon=folium.DivIcon(
                # Set icon size to the minimum needed for the text
                icon_size=(label_width, label_height),  # Let the size be content-dependent
                icon_anchor=(0, 0),  # Anchor the icon based on the centroid
                html=f'<div style="font-size: 12pt; text-align: center; margin: 0 auto; background-color: white; padding: 2px 4px; border: 1px solid black; display: inline-block;">{traitement}</div>'
            )
        ).add_to(map)

    # Add Esri map as a background
    esri_url = 'https://services.arcgisonline.com/arcgis/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}'
    esri_tile_layer = folium.TileLayer(tiles=esri_url, attr='Esri', name='Esri Imagery', overlay=True)
    esri_tile_layer.add_to(map)

    # Save the map as an HTML file
    html_result_folder = base_directory.joinpath('results', 'parcel_visualisation', 'html')
    html_result_folder.mkdir(parents=True, exist_ok=True)  # Create folder if it doesn't exist
    html_file_path = html_result_folder.joinpath(f'{folder_name}_map.html')
    map.save(str(html_file_path))

    # Create PNG image
    png_result_folder = base_directory.joinpath('results', 'parcel_visualisation', 'png')
    png_result_folder.mkdir(parents=True, exist_ok=True)  # Create folder if it doesn't exist
    png_file_path = png_result_folder.joinpath(f'{folder_name}_map.png')
    
    # Generate PNG using selenium webdriver for full HTML element capture
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)
    driver.set_window_size(1600, 1200)  # choose a resolution that fits your map
    driver.get('file:///' + str(html_file_path))
    time.sleep(5)  # Wait for map tiles to load
    driver.save_screenshot(str(png_file_path))
    driver.quit()

## Map Generation Execution

Run the code below to iterate over each subfolder in the specified base directory and create maps for each set of parcel shapefiles.

In [32]:
if __name__ == '__main__':
    base_dir = Path(os.path.dirname(os.path.realpath("__file__")))  # Define base directory
    base_folder = base_dir.joinpath('field_information/Shapefiles')

    # Get a list of all subfolders in the base folder
    subfolders = [f.path for f in os.scandir(base_folder) if f.is_dir()]

    # Iterate over the subfolders and create maps
    for subfolder in subfolders:
        create_map(os.path.basename(subfolder))